In [1]:
import pyrealsense2 as rs
import matplotlib.pyplot as plt
import numpy as np
import imageio
import time
import cv2
import glob
import os
import multiprocessing as mp
from itertools import repeat

def extract_tiff_from_bag(filename, destination):
    try:
        pipeline = rs.pipeline()
        config = rs.config()
        config.enable_device_from_file(filename, False)
        pipeline.start(config)
    except RuntimeError:
        return




    decimation = rs.decimation_filter()
    decimation.set_option(rs.option.filter_magnitude, 1)

    spatial = rs.spatial_filter()
    spatial.set_option(rs.option.filter_magnitude, 5)
    spatial.set_option(rs.option.filter_smooth_alpha, 1)
    spatial.set_option(rs.option.filter_smooth_delta, 50)
    spatial.set_option(rs.option.holes_fill, 2)

    i = 0
    stop = True
    while True:
        try:
            frames = pipeline.wait_for_frames(1000)
            depth_frame = frames.get_depth_frame()

            decimated_depth = decimation.process(depth_frame)
            filtered_depth = spatial.process(decimated_depth)
            depth_image = np.asanyarray(filtered_depth.get_data())
            imageio.imwrite(destination+f'{i}.tif', depth_image, 'TIFF')
            time.sleep(0.01)
            i+=1
            print(destination+f'{i}.tif')

        except RuntimeError:
            pipeline.stop()
            stop = False
            break

    if stop == True:
        pipeline.stop()
    return

def extract_all_tiff_from_folder(filenames, destinations):
    for i in range(0, len(filenames)):
        extract_tiff_from_bag(filenames[i], destinations[i])

In [5]:
def main():
    time1 = time.time()
    filename = 'Videos'
    filenames = glob.glob(filename+'\\unprocessed\\*.bag')
    #print(len(filenames))
    print(filenames)
    destinations = []
    %ls

    dest_dir = filename+'\\unprocessed_tif'

    try:
        os.mkdir(dest_dir)
    except FileExistsError:
        pass

    for i in range(0, len(filenames)):
        #os.chdir(dest_dir)
        try:
            os.mkdir(dest_dir+'\\'+os.path.basename(filenames[i])[0:-4])
        except FileExistsError:
            pass
        destinations.append('\\Users\\zacha\\Documents\\Precision Ranching Code\\'+dest_dir+'\\'+os.path.basename(filenames[i])[0:-4])
        #os.chdir('..')


        # num_cpu = mp.cpu_count()


    print(destinations)
    # for w in range(num_cpu):
    #     p = mp.Process(target=extract_all_tiff_from_folder, args=(filenames, destinations))
    #     processes.append(p)
    #     p.start()
    #
    # for p in processes:
    #     p.join()
    pool = mp.Pool(16)
    m = mp.Manager()
    q = m.Queue()
    #zip(filenames, destinations)
    #pool_tuple = [(x, q) for x in range(1, no_pages)]
    #for i in range(0, len(filenames)):
    pool.starmap(extract_tiff_from_bag, zip(filenames, destinations))
    pool.close()

    time2 = time.time()
    print('Completed in ', time2-time1, ' seconds')
if __name__=='__main__':
    mp.freeze_support()
    main()

['Videos\\unprocessed\\unprocessed_vid_1.bag', 'Videos\\unprocessed\\unprocessed_vid_2.bag', 'Videos\\unprocessed\\unprocessed_vid_3.bag']
 Volume in drive C has no label.
 Volume Serial Number is 7053-FE84

 Directory of c:\Users\zacha\Documents\Precision Ranching Code

10/20/2021  07:46 PM    <DIR>          .
10/20/2021  07:46 PM    <DIR>          ..
10/13/2021  01:47 PM    <DIR>          .ipynb_checkpoints
10/01/2021  10:10 AM            17,156 .py
10/01/2021  10:20 AM    <DIR>          __pycache__
04/14/2021  09:34 AM        94,556,237 20210414_093448.bag
04/14/2021  09:36 AM        95,130,418 20210414_093626.bag
10/19/2021  09:50 PM             7,749 auto_camera.py
10/04/2021  11:43 AM           327,921 Automated BCS.ipynb
10/14/2021  10:47 AM           297,312 Bag Conversion Code.ipynb
10/01/2021  10:26 AM             4,724 BCS_functions.py
08/31/2021  10:12 AM             8,956 Camera Code v2.ipynb
01/27/2021  03:01 PM             5,833 Camera Code v2.py
02/01/2021  12:36 PM    